In [1]:
import pandas as pd
from os import listdir
import re
import numpy as np
import requests
import time
import json
from datetime import date
import datetime
from datetime import timedelta
import pyreadr

In [2]:
from tqdm import tqdm

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [4]:
def load_data_fact_checks():
    fc = pd.read_json('../../../../data/1-factchecks/2-clean_factchecks/factchecks_data.json')
    fc = fc[fc['type_factcheck'] == 'social_media']
    return(fc)

def clean_label(x):
    if x in ['Incorrect','Understated, exaggerated and incorrect',
            'Seven incorrect, three correct, three unproven and one mostly correct']:
        return('fake')
    elif x in ['Mostly correct',
               'A range of verdicts ranging from misleading to unproven and correct',
              'Checked', 'Misleading']:
        return('misleading')
    else:
        return('true')
def load_africa_data():
    africa = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/social/0-prepare_data/facebook_africa.xlsx')[['clean_link', 
    'id_post', 
    'id_desinformacion', 
    'Verdict']].rename(columns = {'clean_link':'link_desinformacion'})
    africa['link_post'] = africa.link_desinformacion
    africa['label_desinformacion'] = africa['Verdict'].apply(lambda x: clean_label(x))
    return(africa)
def load_social_data():
    social = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/social/2-id_data/virality_fake_posts.xlsx')

    social_misinformation = social[['id_post', 'id_desinformacion', 'link_desinformacion', 'link_crowdtangle', 
       'label_desinformacion', 'id_chequeo']].rename(columns = {'link_crowdtangle':'link_post', 
                                                               'id_chequeo':'id_factcheck'})
    
    return(social_misinformation)

In [5]:
def obtian_statistics_virality(df_crowd):
    date_post = df_crowd.columns[-2:][0]
    
    reactions = df_crowd[['ID', 'Score Date (GMT)', 'Timestep', 
               'Likes', 'Comments', 'Shares', 'Loves', 'Wows', 'Hahas', 'Sads', 'Angrys', 'Cares', 'Reactions', 'Total Views', 'Overperforming(+)/Underperforming(-) Score']]
    
    link_crowdtangle = df_crowd.columns[-1:][0]
    user_desinformacion = re.sub('https://www.facebook.com/', '', link_crowdtangle).split('/')[0]
    link_user_desinformacion = 'https://www.facebook.com/' + user_desinformacion
    
    reactions['date_post'] = date_post
    reactions['link_crowdtangle'] = link_crowdtangle
    reactions['user_desinformacion'] = user_desinformacion
    reactions['link_user_desinformacion'] = link_user_desinformacion
    
    return(reactions)

In [6]:
def match_virality_timeseries(data_id):
    path_crowdtangle = '/Users/cblanesg/Desktop/timeline_crowdtangle/'

    id_engage = []
    file = []
    for i in listdir(path_crowdtangle):
        if i == '.DS_Store':
            continue
        else:
            id_engage.append(re.sub(' [(]1[)]', '', re.sub('.csv', '', i.split('_')[-1:][0])))
            file.append(i)

    ids_posts = []
    path_file = []
    id_desinformacion = []
    for i, x in zip(id_engage, file):
        if i in list(data_id.id_post):
            ids_posts.append(i)
            id_desinformacion.append(list(id_data[id_data['id_post'] == i].id_desinformacion)[0])
            path_file.append(x)
        else:
            for j in list(data_id.link_desinformacion):
                if any(re.findall(i, j)):
                    ids_posts.append(i)
                    id_desinformacion.append(list(id_data[id_data['link_desinformacion'] == j].id_desinformacion)[0])
                    path_file.append(x)
                else:
                    continue
                    
    matching_data = pd.DataFrame({'id_post':ids_posts, 
                                 'id_desinformacion':id_desinformacion, 
                                 'name_file':path_file})
    return(matching_data)
    

In [7]:
def changeid(link, id_post):
    if any(re.findall('permalink[.]php[?]story', link)):
        id_clean = link.split('story_fbid=')[1].split('&id=')[0]
        return(id_clean)
    else:
        return(id_post)

In [8]:
def clean_date_publication(x):
    str_date = x.split(' ')[0]
    return(datetime.date(int(x.split(' ')[0].split('-')[0]), 
                         int(x.split(' ')[0].split('-')[1]),
                        int(x.split(' ')[0].split('-')[2])))

In [9]:
def add_missing_timesteps(temp):
    missing_rows = []
    for i in list(range(-16, 16)):
        if i not in temp.days_since_factcheck.unique():
            row = []
            timestep = list(temp.date_factcheck)[0] + timedelta(i)
            
            
            row.append(list(temp.id_post_desinformacion)[0])
            row.append(list(temp.id_desinformacion)[0])
            row.append(list(temp.date_publication)[0])
            row.append(timestep)
            row.append((timestep - list(temp.date_publication)[0]).days)
            row.append(list(temp.date_factcheck)[0])
            row.append(i)
            row.append(list(temp.facebook_partnership_date)[0])
            row.append(list(temp.link_user_desinformacion)[0])
            row.append(list(temp.link_desinformacion)[0])
            row.append(list(temp.organizacion)[0])
            row.append(list(temp.pais)[0])
            row.append(list(temp.id_factcheck)[0])
            row.append(np.nan)
            row.append(np.nan)
            row.append(np.nan)
            row.append(np.nan)
            
            m_row = pd.DataFrame([row], columns = list(temp.columns))
            missing_rows.append(m_row)
    temp_clean = pd.concat([temp, pd.concat(missing_rows)])
    temp_clean = temp_clean[temp_clean['days_since_publication'] >= 0]
    return(temp_clean)

## Load Data And Apply Functions

In [10]:
id_data = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/social/0-prepare_data/id_data_virality5.xlsx')
id_data['id_post'] = id_data.apply(lambda x: changeid(x['link_desinformacion'], x['id_post']), axis = 1)

In [11]:
existing_engagement = match_virality_timeseries(id_data)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/cblanesg/Desktop/timeline_crowdtangle/'

In [ ]:
timeseries_desinformacion = []

for i in tqdm(range(0, len(existing_engagement))):
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/social/1-engagement/facebook/timeline_crowdtangle/'

    inner_panel = obtian_statistics_virality(pd.read_csv(path + existing_engagement.loc[i]['name_file']))
    post = existing_engagement.loc[i]['id_post']
    id_desinformacion = existing_engagement.loc[i]['id_desinformacion']
    file_name = existing_engagement.loc[i]['name_file']
    
    inner_panel['file_name'] = file_name
    inner_panel['id_desinformacion'] = id_desinformacion
    inner_panel['post'] = post
    
    timeseries_desinformacion.append(inner_panel)

timeseries_desinformacion = pd.concat(timeseries_desinformacion)

In [ ]:
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/social/3-panel_disaggreagted/panel_facebook_posts.json', 'w') as f:
    json.dump(timeseries_desinformacion.to_dict('records'), f)

## Clean Panel 

In [20]:
data = pd.read_json('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/social/3-panel_disaggreagted/panel_facebook_posts.json')

In [25]:
factchecks_id = load_social_data()[['id_desinformacion', 'id_factcheck']].dropna().drop_duplicates()
factchecks = load_data_fact_checks()

In [26]:
factchecks

,id_factcheck,type_factcheck,organizacion,pais,facebook_partnership_date,facebook_partnership,titulo_chequeo,link_chequeo,fecha_chequeo,link_facebook_factcheck,date_factcheck_facebook,dummy_facebook_date,difference_dates,date_factcheck_final,fecha_factcheck
0,2c51a443-dcfe-3fdc-b901-79566bb18dd0,social_media,larepublica,peru,2020-06-01,1,cafe haiti es falso que el cafe haiti de miraf...,https://larepublica.pe/verificador/2020/08/05/...,2020-05-08,None,None,0,NaN,2020-05-08,2020-08-05
1,1718b0ab-d593-3704-a78a-bb095d6ecb14,social_media,larepublica,peru,2020-06-01,1,areas de aislamiento es falsa la publicacion d...,https://larepublica.pe/verificador/2020/08/05/...,2020-05-08,None,None,0,NaN,2020-05-08,2020-08-05
2,3c167832-8e73-330a-ab8a-19b219f79dfc,social_media,larepublica,peru,2020-06-01,1,dioxido de cloro es imprecisa la publicacion d...,https://larepublica.pe/verificador/2020/08/04/...,2020-04-08,None,None,0,NaN,2020-04-08,2020-08-04
3,656ae72f-a364-353d-813a-dd24472479a6,social_media,larepublica,peru,2020-06-01,1,5g es falso que foto es de vacuna china con ar...,https://larepublica.pe/verificador/2020/08/03/...,2020-03-08,None,None,0,NaN,2020-03-08,2020-08-03
4,6fcce096-cf30-3f7c-ae86-591590d024e6,social_media,larepublica,peru,2020-06-01,1,coronavirus es impreciso afirmar que colegio m...,https://larepublica.pe/verificador/2020/08/03/...,2020-03-08,None,None,0,NaN,2020-03-08,2020-08-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148,78619835-70c7-3138-a980-2a1d925af857,social_media,africa_check,africa,2019-08-15,1,are sa whites really being killed flies why st...,https://africacheck.org/fact-checks/reports/ar...,2013-06-24,None,None,0,NaN,2013-06-24,2013-06-24
4149,51dcaef1-9539-30dd-8ade-f745007b1a65,social_media,africa_check,africa,2019-08-15,1,south africa largest recipient asylum seekers ...,https://africacheck.org/fact-checks/reports/so...,2013-07-11,None,None,0,NaN,2013-07-11,2013-11-07
4163,8c0db5f3-caf0-37a6-b35e-e51e8ba7ace8,social_media,africa_check,africa,2019-08-15,1,how south africas department basic education m...,https://africacheck.org/fact-checks/reports/ho...,2013-10-09,None,None,0,NaN,2013-10-09,2013-09-10
4171,5011598d-6ac8-3b26-84a1-12d7be008e7b,social_media,africa_check,africa,2019-08-15,1,zuma correct claim million households were ele...,https://africacheck.org/fact-checks/reports/zu...,2013-11-13,None,2013-11-13,1,0.0,2013-11-13,2013-11-13


In [27]:
complete_fc = pd.merge(left = factchecks, 
        right = factchecks_id, 
        on = 'id_factcheck', how = 'left')

In [28]:
data['date_publication'] = data['date_post'].apply(lambda x: clean_date_publication(x))
data['date_timestep'] = data['Score Date (GMT)'].apply(lambda x: clean_date_publication(x))
data['days_since_publication'] = data.apply(lambda x: (x['date_timestep'] - x['date_publication']).days, axis = 1)

In [29]:
social = pd.merge(left = data, 
        right = complete_fc, 
        on = 'id_desinformacion', 
        how = 'left')
social = social[~social['fecha_factcheck'].isna()]
social['date_factcheck'] = social['fecha_factcheck'].apply(lambda x: datetime.date(int(x.split('-')[0]), 
                                                                               int(x.split('-')[1]), 
                                                                               int(x.split('-')[2])))

In [30]:
social['days_since_factcheck'] = social.apply(lambda x: (x['date_timestep'] - x['date_factcheck']).days, axis = 1)

In [31]:
social['loveCount'] = pd.to_numeric(social.Loves)
social['hahaCount'] = pd.to_numeric(social.Wows)
social['wowCount'] = pd.to_numeric(social.Hahas)
social['sadCount'] = pd.to_numeric(social.Sads)
social['angryCount'] = pd.to_numeric(social.Angrys)
social['thankfulCount'] = pd.to_numeric(social.Cares)

In [32]:
social['reactions'] = social.apply(lambda x:
                                      x['Loves'] + x['Wows'] + x['Hahas'] +
                                      x['Sads'] + x['Angrys'] + 
                                      x['Cares'], axis = 1)

In [33]:
input_panel = social[['ID', 
       'id_desinformacion', 
        'date_publication', 
        'date_timestep', 
       'days_since_publication' ,
                      'date_factcheck', 
                      'days_since_factcheck',
       'facebook_partnership_date', 
       'link_user_desinformacion', 
       'link_crowdtangle',
       'organizacion', 
       'pais', 'id_factcheck',
                      'Likes', 
                     'Comments', 'Shares', 
                     'reactions']].rename(columns = {'ID':'id_post_desinformacion', 
                                             'link_crowdtangle':'link_desinformacion', 
                                                    'Likes':'likes', 
                                                    'Comments':'comments', 
                                                    'Shares':'shares'})

## Make Panel And interpolate

In [34]:
df_all = []

In [35]:
for i in tqdm(input_panel.id_post_desinformacion.unique()):
    temp = input_panel[input_panel['id_post_desinformacion'] == i]
    temp = temp.sort_values('days_since_publication')
    temp = temp.drop_duplicates('days_since_publication', keep= 'last')
    df_all.append(add_missing_timesteps(temp))

100%|██████████| 307/307 [00:30<00:00, 10.06it/s]


In [36]:
df_final_aggregated = pd.concat(df_all)

In [37]:
panel_input = df_final_aggregated[['date_timestep', 'id_desinformacion', 'date_publication',
       'days_since_publication', 'date_factcheck', 'days_since_factcheck',
       'id_post_desinformacion', 'id_factcheck', 'link_desinformacion',
       'facebook_partnership_date', 'organizacion', 'pais', 
                                'likes', 'shares', 'comments', 'reactions']]

In [38]:
panel_interpolate = []
for i in tqdm(panel_input.id_post_desinformacion.unique()):
    df = panel_input[panel_input['id_post_desinformacion'] == i].sort_values('days_since_publication')
    
    df['approx_likes'] = df.likes.interpolate()
    df['approx_shares'] = df.shares.interpolate()
    df['approx_comments'] = df.comments.interpolate()
    df['approx_reactions'] = df.reactions.interpolate()
    panel_interpolate.append(df)   

100%|██████████| 307/307 [00:01<00:00, 164.07it/s]


In [39]:
df_interpolate = pd.concat(panel_interpolate)

In [40]:
df_interpolate = df_interpolate[abs(df_interpolate['days_since_factcheck']) <=15]

In [41]:
df_interpolate['approx_reactions'] = pd.to_numeric(df_interpolate.approx_reactions)
df_interpolate['approx_likes'] = pd.to_numeric(df_interpolate.approx_likes)
df_interpolate['approx_shares'] = pd.to_numeric(df_interpolate.approx_shares)
df_interpolate['approx_comments'] = pd.to_numeric(df_interpolate.approx_comments)

In [42]:
df_interpolate['approx_interactions'] = df_interpolate.apply(lambda x: x['approx_likes'] + 
                                                            x['approx_comments'] + x['approx_shares'] + 
                                                            x['approx_reactions'], axis = 1)

In [43]:
df_interpolate['growth_reactions'] = df_interpolate.groupby(['id_post_desinformacion']).approx_reactions.pct_change()
df_interpolate['growth_likes'] = df_interpolate.groupby(['id_post_desinformacion']).approx_likes.pct_change()
df_interpolate['growth_shares'] = df_interpolate.groupby(['id_post_desinformacion']).approx_shares.pct_change()
df_interpolate['growth_comments'] = df_interpolate.groupby(['id_post_desinformacion']).approx_comments.pct_change()
df_interpolate['growth_interactions'] = df_interpolate.groupby(['id_post_desinformacion']).approx_interactions.pct_change()

In [63]:
distance = df_interpolate[['id_desinformacion','days_since_factcheck', 'days_since_publication']]
distance = distance[distance['days_since_factcheck'] == 0]

In [64]:
distance[distance['days_since_publication'] > 100]

,id_desinformacion,days_since_factcheck,days_since_publication
0,c2403c7e-c600-3770-b5b2-dbdb4c14958d,0,268
0,99d926d6-c66b-31cd-9d97-17982527e825,0,599
0,2301869f-9c58-3eb1-8e69-c798151fb10d,0,125
0,672784ca-02b5-11eb-87de-acde48001122,0,772
0,57ebe593-14b4-3b33-bf17-480a71067108,0,226
0,4727acfe-e55a-328e-a86b-8e31c05d642f,0,270
0,cc35032f-03bc-367e-84f5-01df600dc628,0,371
0,6727879a-02b5-11eb-87de-acde48001122,0,373
0,9291ab2a-bf71-39f8-8d12-2a6be8c083d2,0,186
0,c5c5f6b5-2bc5-3d29-a2b4-82e7b6bbd712,0,154


In [65]:
pyreadr.write_rdata('../../../../data/4-panel_data/social/panel_interpolate.RData', df_interpolate, df_name="misinformation_social_panel")